In [99]:
from PIL import Image

In [107]:
import skimage.measure
def pooling_data(Iftrain, data, basepixel, pooling_size):
    
    #print pd.DataFrame(data)

    #大于某个像素值 ==> 255
    if Iftrain == True:
        if basepixel > 0:
            data[data> basepixel] =  255
            data[data<= basepixel] = 0
            
    #压缩矩阵操作,取平均值，你可以自己写函数
    pooled_data = skimage.measure.block_reduce(data,pooling_size,  np.mean)
    return pooled_data

In [109]:
import os


def save_to_image(Iftrain, pixels,basepixel,size, i):
    MINST = ""
    if Iftrain == True:
        MINST = "TRIAN_DATA"
    else:
        MINST = "TEST_DATA"
        
    image = Image.new('L',pixels.shape)
    columns = pixels.shape[0]
    rows = pixels.shape[1]
    
    for x in xrange(columns):
            for y in xrange(rows):
                    pix = pixels[y][x]
                    image.putpixel((y, x), int(pixels[y][x]))
    
    
    directory = "pooling/"+MINST+"/"+"basepixel_"+str(basepixel) +"/"+ str(columns)+"_"+str(rows)
    print directory+str(i)
    #print pd.DataFrame(pixels)
    
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    image.save(directory+"/"+str(i) + '.png')

In [110]:
def save_to_numpy(Iftrain, train,  label, basepixel, size):
    MINST = ""
    if Iftrain == True:
        MINST = "TRIAN_DATA"
    else:
        MINST = "TEST_DATA"
        
    directory = MINST+"/"+"basepixel_"+str(basepixel) +"/"+ str(size[0])+"_"+str(size[1])+"/"
    #print pd.DataFrame(pixels)
    
    if not os.path.exists(directory):
        os.makedirs(directory)
    ###################################################
    np.save(directory+'MINST_train.npy', train)
    test = np.load(directory+'MINST_train.npy')
    print np.all(train == test)  
    ###################################################
    np.save(directory+"MINST_label.npy", label)
    label_test = np.load(directory+'MINST_label.npy')
    print np.all(label == label_test)  
    ##################################################
    
    print train.shape
    with open(directory+'MINST_train.txt','wb') as f:
        for row in train:
             np.savetxt(f,np.transpose(row),delimiter=',',fmt = "%03d", footer='\n')
                
    with open(directory+'MINST_label.txt','wb') as f:
        for row in label:
             np.savetxt(f,row,delimiter=',',fmt = "%01d", footer='\n')            
    #x = np.loadtxt(directory+'MINST_train.txt',delimiter=',')
    #print np.all(x == train)

In [113]:
from PIL import Image
import struct
import numpy as np
import pandas as pd
import math

def deal_image(train_filename, label_filename, pooling_size_list, Debug_num = 0,Iftrain = True):
    
    ##############################################################################################
    train_file = open(train_filename, 'rb')
    buf = train_file.read()
    print "buf", len(buf)
    train_file.close()
    index = 0
    magic, images, rows, columns = struct.unpack_from('>IIII' , buf , index)
    print "magic", magic, "images", images, rows, columns
    index += struct.calcsize('>IIII')
    if Debug_num == 0:
        images_num = images
    else:
        images_num = Debug_num    
    ################################################################################################
    label_f = open(label_filename, 'rb')
    label_buf = label_f.read()
    label_index = 0
    struct.unpack_from('>II' , label_buf, label_index)
    label_index += struct.calcsize('>II')
    
    for size in pooling_size_list:
        for basepixel in range(0, 256, 25):
            
            pooled_size = (int(math.ceil(columns*1.0/size[0])) , int(math.ceil(rows*1.0/size[1])))
            print pooled_size
            
            
            ###################################################################
            np_to_save = np.zeros((images_num,  pooled_size[0], pooled_size[1]))
            label_to_save = np.zeros((images_num,1, 10))
            ###################################################################
            index = 0
            index += struct.calcsize('>IIII')
            ##############################
            ##############################
            label_index = 0
            label_index += struct.calcsize('>II')
            ##############################
            
            for i in xrange(images_num):
                # for debug
                

                normal_np = np.zeros((columns, rows))
                label_np = np.zeros(10)
                label_number = (struct.unpack_from('>B', label_buf, label_index)[0])
                
                label_index += struct.calcsize('>B')

                print label_number
                label_np[label_number] = 1
                #冗余的代码
                for x in xrange(columns):
                  for y in xrange(rows):
                    normal_np[y][x] = int(struct.unpack_from('>B', buf, index)[0])
                    index += struct.calcsize('>B')
                
                

                #print normal_np.shape
                print np_to_save[i].shape
                pooled_data = pooling_data(Iftrain, normal_np, basepixel, size)
                ###############################################################
                np_to_save[i] =  pooled_data
                label_to_save[i] =label_np
                #print np_to_save[i]
                ################################################################
                save_to_image(Iftrain, pooled_data,basepixel,size, i)
            print  "labeltosave"
            print  label_to_save
            print  np_to_save[0]
            save_to_numpy(Iftrain, np_to_save, label_to_save, basepixel, pooled_size) 

In [116]:
#设置要池化的数据块大小
pooling_size_list = [(1,1), (2,2)]
#if Debug_num == 0, then save all images, Debug_num >0, save Debug_num images
deal_image("train-images.idx3-ubyte", "train-labels.idx1-ubyte",pooling_size_list,Debug_num=10,Iftrain=True)
deal_image("t10k-images.idx3-ubyte","t10k-labels.idx1-ubyte", pooling_size_list,Debug_num=10,Iftrain=False)


buf 47040016
magic 2051 images 60000 28 28
(28, 28)
5
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_280
0
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_281
4
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_282
1
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_283
9
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_284
2
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_285
1
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_286
3
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_287
1
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_288
4
(28, 28)
pooling/TRIAN_DATA/basepixel_0/28_289
labeltosave
[[[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]]

 [[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]]

 [[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]]

 [[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]]

 [[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]]

 [[ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]]

 [[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.  1.  0.  0.  

In [ ]:
def read_to_numpy(basepixel, size):
    directory = "MINST_train/"+"basepixel_"+str(basepixel) +"/"+ str(size[0])+"_"+str(size[1])+"/"
    
    #print pd.DataFrame(pixels)
    print directory
#     if not os.path.exists(directory):
#         os.makedirs(directory)
    
    test = np.load(directory+'MINST_train.npy')
    
    img = Image.fromarray(test[1], 'L')
    img.show()
    print pd.DataFrame(test[1])

In [37]:
dic = {0:"0000000001", 1:"0000000010",2:"0000000100",3:0000001000,4:0000010000,5:0000100000,6:0001000000,7:0010000000,8:0100000000,9:1000000000}
dic[2]

64

In [21]:
import numpy as np
from PIL import Image

# Creates a random image 100*100 pixels
mat = np.random.random((100,100))

# Creates PIL image
img = Image.fromarray(mat, 'L')
img.show() 

In [85]:
label_index = 0
label_index += struct.calcsize('>9B')
buf2 = open("train-labels.idx1-ubyte", "rb").read()
print struct.unpack_from('>B', buf2, 8)


(5,)


In [38]:
np.zeros(10)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])